In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
sns.set_context('poster')

In [71]:
import json
import os
import glob
import re

In [72]:
import tarfile

In [73]:
!ls

01_Analysis.ipynb       benchmark.sh            run_tuplex.py
01_CloudWatchLogs.ipynb log.txt                 tuplex-results.tar.gz
Dockerfile              results


In [74]:
def decode(content):
    lines = content.split('\n')
    
    stats = list(filter(lambda L: L.startswith('{"startupTime"'), lines))
    stats = json.loads(stats[0])
    
    
    task_lines = filter(lambda L: 'LAMBDA task' in L, lines)
    
    def lam_info(line):
        m = re.search('(\d\.\d+)s', line)
        time = float(m[1])
        
        uid = line[line.find('id: ')+4:]
        uid = uid[:uid.find(']')]
        
        
        return {'time': time, 'id': uid}
    
    tasks = map(lam_info, task_lines)
    df_tasks = pd.DataFrame(tasks)
    
    # how many unique containers were launched?
    num_unique_containers = len(df_tasks['id'].unique())
    
    # what was the mean execution time per container?
    # (distribution)
    task_time_mean = df_tasks['time'].mean()
    task_time_std = df_tasks['time'].std()
    
    container_reuse_min = int(df_tasks.groupby('id').count().min())
    container_reuse_max = int(df_tasks.groupby('id').count().max())
    
    # could also do some fancier stats here, i.e. histogram/binning on reuse!
    
    row = stats
    row.update({'num_unique_containers' : num_unique_containers,
           'task_time_mean' : task_time_mean,
           'task_time_std': task_time_std,
           'num_tasks':len(df_tasks),
           'container_reuse_min' : container_reuse_min,
           'container_reuse_max': container_reuse_max})
    return row

In [75]:
rows = []
with tarfile.open('tuplex-results.tar.gz') as tf:
    for name in filter(lambda name: name.endswith('.txt') and name.startswith('run'), tf.getnames()):
        fp = tf.extractfile(name)
        if fp is not None:
            content = fp.read().decode('utf-8')
            
            _, run, _, size, mode = name.split('-')
            run = int(run)
            size = int(size.replace('mb', ''))
            mode = mode.replace('.txt', '')
            run, size, mode
            
            row = {'run' : run, 'size' : size, 'mode':mode, 'framework':'tuplex'}
            
            row.update(decode(content))
            rows.append(row)
#             row = json.loads(content)
#             row['name'] = name
#             rows.append(row)

In [77]:
df = pd.DataFrame(rows)

In [80]:
df.groupby(['size', 'mode']).mean()

run  startupTime    jobTime  num_unique_containers  \
size  mode                                                       
3000  cold  3.0     0.064736  27.108625                  102.8   
      warm  3.0     0.070978  25.536860                  100.0   
5000  cold  3.0     0.070534  25.155967                  104.2   
      warm  3.0     0.072199  23.751910                  102.6   
10000 cold  3.0     0.068485  25.577497                  105.4   
      warm  3.0     0.080214  25.127351                  102.2   

            task_time_mean  task_time_std  num_tasks  container_reuse_min  \
size  mode                                                                  
3000  cold        4.766631       0.339352      400.0                  2.0   
      warm        4.973633       0.474070      400.0                  3.0   
5000  cold        4.797372       0.392640      400.0                  1.8   
      warm        4.748262       0.285700      400.0                  2.6   
10000 cold        4.759836       0.278665      400.0                  2.0   
      warm        4.762591       0.441136      400.0                  2.6   

            container_reuse_max  
size  mode                       
3000  cold                  4.6  
      warm                  5.0  
5000  cold                  5.0  
      warm                  4.6  
10000 cold                  4.4  
      warm                  5.0

In [81]:
# other interesting stats:
# => max concurrency (via cloudwatch)